This contains the tools for publishing a public version of first reading resolutions.

This should be used after plenary. 

It will create a folder inside Google Drive labeled 'For campus feedback' with copies of all first reading resolutions

https://developers.google.com/docs/api/quickstart/python

In [ ]:
from googleapiclient.discovery import build

In [ ]:
import os
if os.path.exists('../token-1.json'):
    print('j')


In [ ]:

import API.CredentialsManager

In [ ]:
%cd ~/Dropbox/ResolutionManager

from ResolutionManager import environment as env
from ResolutionManager.API.CredentialsManager import CredentialsManager
from ResolutionManager.Repositories.DocumentRepository import DocumentRepository
from ResolutionManager.Repositories.FileRepository import FileRepository
from ResolutionManager.Repositories.ResolutionTemplateRespository import ResolutionTemplateRepository
import ResolutionManager.executables.make_folders_for_plenary as make_folders
import ResolutionManager.executables.copy_first_readings_for_feedback as copy_first_reads
import ResolutionManager.executables.create_resolution_from_template as create_resolution
from ResolutionManager.Models.Plenaries import Plenary
from ResolutionManager.Models.Committees import Committee
from ResolutionManager.Models.Resolutions import Resolution

# import environment as env


In [ ]:
resolution_name = "Opposing the existence of the CO"
resolution_number = 3456
committee = Committee('Faculty Affairs', 'FA')
cosponsors = [ Committee('Academic Affairs', 'AA')]
resolution = Resolution(resolution_number, resolution_name, committee, cosponsors)


plenary = Plenary(year=2023,
                      month='September',
                      thursday_date=12,
                      friday_date=14,
                      first_reading_folder_id='1sv_4BUV5fk6Kcjss8HeSCJWnsLZJVpKC'
                      )


In [ ]:
committee.__dict__

In [ ]:
resolution.__dict__

# Build folder structure for plenaries

In [ ]:
make_folders.main()

# Create resolution

# Create public feedback versions